In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
cd '/content/drive/My Drive/Rondom_Forests'

/content/drive/My Drive/Rondom_Forests


In [3]:
ls

pima-indians-diabetes.data.csv  random_forests.ipynb


In [4]:
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv

--2021-04-18 16:33:13--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv.1’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0.002s  

2021-04-18 16:33:13 (12.6 MB/s) - ‘pima-indians-diabetes.data.csv.1’ saved [23278/23278]



In [5]:
import pandas as pd
# list for column headers
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
# open file with pd.read_csv
df = pd.read_csv("https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv", names=names)
print(df.shape)
# print head of data set
print(df.head())

(768, 9)
   preg  plas  pres  skin  test  mass   pedi  age  class
0     6   148    72    35     0  33.6  0.627   50      1
1     1    85    66    29     0  26.6  0.351   31      0
2     8   183    64     0     0  23.3  0.672   32      1
3     1    89    66    23    94  28.1  0.167   21      0
4     0   137    40    35   168  43.1  2.288   33      1


In [6]:
X = df.drop('class', axis=1)
y = df['class']

In [7]:
# split into test, train data
from sklearn.model_selection import train_test_split
# implementing train-test-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=66)

In [8]:
# create the random forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_predict = rfc.predict(X_test)

In [9]:
# Evaluating Performance
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

In [10]:
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')

In [11]:
rfc_cv_score

array([0.78333333, 0.8362963 , 0.81814815, 0.72888889, 0.81740741,
       0.84777778, 0.87037037, 0.9037037 , 0.78846154, 0.86      ])

In [12]:
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[148  28]
 [ 32  46]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       176
           1       0.62      0.59      0.61        78

    accuracy                           0.76       254
   macro avg       0.72      0.72      0.72       254
weighted avg       0.76      0.76      0.76       254



=== All AUC Scores ===
[0.78333333 0.8362963  0.81814815 0.72888889 0.81740741 0.84777778
 0.87037037 0.9037037  0.78846154 0.86      ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.8254387464387463


In [13]:
# Tuning Hyperparameters
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
# number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# number of features at every split
max_features = ['auto', 'sqrt']

# max depth
max_depth = [int(x) for x in np.linspace(100, 500, num = 11)]
max_depth.append(None)
# create random grid
random_grid = {
 'n_estimators': n_estimators,
 'max_features': max_features,
 'max_depth': max_depth
 }
# Random search of parameters
rfc_random = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the model
rfc_random.fit(X_train, y_train)
# print results
print(rfc_random.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   52.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.6min finished


{'n_estimators': 800, 'max_features': 'sqrt', 'max_depth': None}


In [14]:
rfc = RandomForestClassifier(n_estimators=600, max_depth=300, max_features='sqrt')
rfc.fit(X_train,y_train)
rfc_predict = rfc.predict(X_test)
rfc_cv_score = cross_val_score(rfc, X, y, cv=10, scoring='roc_auc')
print("=== Confusion Matrix ===")
print(confusion_matrix(y_test, rfc_predict))
print('\n')
print("=== Classification Report ===")
print(classification_report(y_test, rfc_predict))
print('\n')
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ", rfc_cv_score.mean())

=== Confusion Matrix ===
[[149  27]
 [ 34  44]]


=== Classification Report ===
              precision    recall  f1-score   support

           0       0.81      0.85      0.83       176
           1       0.62      0.56      0.59        78

    accuracy                           0.76       254
   macro avg       0.72      0.71      0.71       254
weighted avg       0.75      0.76      0.76       254



=== All AUC Scores ===
[0.78740741 0.84       0.82185185 0.7362963  0.81851852 0.86333333
 0.85962963 0.90740741 0.80269231 0.85269231]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.828982905982906
